<a href="https://www.kaggle.com/code/proloymondal/medical-image-classification?scriptVersionId=147026191" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from tensorflow.keras.layers import Conv2D, Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image  import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings("ignore")


In [2]:
model = Sequential()

# Conv 01 Layer
model.add(Convolution2D(96, 11,padding = "valid",strides =(4, 4),input_shape = (224, 224, 3),activation = "relu"))

model.add(MaxPooling2D(pool_size = (2, 2),strides = (2, 2),padding="valid"))
model.add(BatchNormalization())



# Conv 02 Layer
model.add(Convolution2D(256, 5,padding = "valid",strides =(1, 1),activation = "relu"))

model.add(MaxPooling2D(pool_size = (2, 2),strides = (2, 2),padding="valid"))
model.add(BatchNormalization())


# Conv 03 Layer
model.add(Convolution2D(384, 3,padding = "valid",strides =(1, 1),activation = "relu"))
model.add(BatchNormalization())


# Conv 04 Layer
model.add(Convolution2D(384, 3,padding = "valid",strides =(1, 1),activation = "relu"))
model.add(BatchNormalization())



# Conv 05 Layer
model.add(Convolution2D(256, 3,padding = "valid",strides =(1, 1),activation = "relu"))

model.add(MaxPooling2D(pool_size = (3, 3),strides = (2, 2),padding="valid"))
model.add(BatchNormalization())


# Flatten Layer
model.add(Flatten())


# Dense 01
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())


# Dense 02
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())


# Dense 03
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())


# Final Layer
model.add(Dense(3, activation="softmax"))

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 23, 23, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 384)         8

In [4]:
model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9, decay = 0.005), 
             loss = "sparse_categorical_crossentropy",
             metrics = ["accuracy"])

In [5]:
datasets_directory = "../input/dogs-and-cats-image-classification/dataset/"

In [6]:
batch_size = 128

In [7]:
train_dataGenerator = ImageDataGenerator(rescale = 1./255,
                                         
                                         shear_range = 0.2, 
                                         zoom_range=0.2,
                                         width_shift_range=0.2,
                                         height_shift_range=0.2, 
                                         fill_mode="nearest"
                                        )

validation_dataGenerator = ImageDataGenerator(rescale = 1./255)


In [8]:
# Load the datasets from the folder
trainDatasets = train_dataGenerator.flow_from_directory(datasets_directory+"/training_set", 
                                                       target_size=(224, 224),
                                                       batch_size=batch_size, 
                                                       class_mode='binary')

validationDatasets = validation_dataGenerator.flow_from_directory(datasets_directory+"/test_set", 
                                                       target_size=(224, 224),
                                                       batch_size=batch_size, 
                                                       class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [9]:
classInformation = trainDatasets.class_indices
print("Class Information: {}".format(classInformation))
classes = list(classInformation)
print("Class Names: {}".format(classes))

Class Information: {'cats': 0, 'dogs': 1}
Class Names: ['cats', 'dogs']


In [10]:
classSamplesT = trainDatasets.samples
print("Training Data Samples: {}".format(classSamplesT))
classSamplesV = validationDatasets.samples
print("Validation Data Samples: {}".format(classSamplesV))

Training Data Samples: 8000
Validation Data Samples: 2000


In [11]:
weghtPath = "best_weights_9.hdf5"
checkPoint = ModelCheckpoint(weghtPath, monitor="val_acc", verbose = 1, save_best_only = True, save_weghts_only = True, mode = "max")
callBacksList = [checkPoint]

In [12]:
history = model.fit(trainDatasets,
                    steps_per_epoch = classSamplesT//batch_size, 
                    validation_data=validationDatasets,
                    epochs=20, 
                    validation_steps=classSamplesV//batch_size, 
                    callbacks=callBacksList)

Epoch 1/20
62/62 [==============================] - 405s 6s/step - loss: 1.4026 - accuracy: 0.4117 - val_loss: 0.9129 - val_accuracy: 0.5432
Epoch 2/20
62/62 [==============================] - 381s 6s/step - loss: 1.1587 - accuracy: 0.4862 - val_loss: 0.9326 - val_accuracy: 0.5510
Epoch 3/20
62/62 [==============================] - 376s 6s/step - loss: 1.0333 - accuracy: 0.5224 - val_loss: 0.8422 - val_accuracy: 0.5969
Epoch 4/20
62/62 [==============================] - 374s 6s/step - loss: 0.9414 - accuracy: 0.5683 - val_loss: 0.7960 - val_accuracy: 0.6146
Epoch 5/20
62/62 [==============================] - 375s 6s/step - loss: 0.9065 - accuracy: 0.5833 - val_loss: 0.7654 - val_accuracy: 0.6255
Epoch 6/20
62/62 [==============================] - 374s 6s/step - loss: 0.8819 - accuracy: 0.5823 - val_loss: 0.7449 - val_accuracy: 0.6359
Epoch 7/20
62/62 [==============================] - 366s 6s/step - loss: 0.8497 - accuracy: 0.6032 - val_loss: 0.7054 - val_accuracy: 0.6641
Epoch 8/20
62